In [1]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import sklearn
import warnings

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import utils
import ml_functions

red_code = '\033[91m'
blue_code = '\033[94m'
green_code = '\033[92m'
yellow_code = '\033[93m'
reset_code = '\033[0m'

C:\Users\NeilB\AppData\Local\Temp\ipykernel_28744\913017747.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from sklearn import (
    linear_model, 
    ensemble,
    tree,
    decomposition, 
    naive_bayes, 
    neural_network,
    svm,
    metrics,
    preprocessing, 
    model_selection, 
    pipeline,
)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

In [3]:
def logistic_regression(X_train, X_test, y_train, y_test):
    model = clf = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000)
    # Entraîner le modèle sur les données d'entraînement
    model.fit(X_train, y_train)

    # Prédire les étiquettes des données de test
    y_pred = model.predict(X_test)

    # Prédire les probabilités des classes positives pour les données de test
    # Prédire les probabilités des classes positives pour les données de test
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
    else:
        # Utiliser la décision de fonction de décision si le modèle ne prend pas en charge predict_proba
        y_prob = model.decision_function(X_test)

    # Calcul des métriques de performance
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)

    # Affichage du rapport de classification
    report = metrics.classification_report(y_test, y_pred)
    # print(report)

    print(f'{green_code}Accuracy :\t{acc}{reset_code}')
    print(f'{green_code}F1 score :\t{f1}{reset_code}')
    print(f'{green_code}AUC :\t\t{auc}{reset_code}')

### Chargement des données

In [4]:
path = "./datasets/movies/movies1000/"
alltxts,alllabs = utils.load_movies(path)

movies_df = pd.DataFrame()
movies_df['text'] = alltxts
movies_df['label'] = alllabs

In [5]:
preprocessed_alltxts = [utils.preprocess(alltxt, True) for alltxt in movies_df.text]

preprocessed_movies_df = pd.DataFrame()
preprocessed_movies_df['text'] = preprocessed_alltxts
preprocessed_movies_df['label'] = alllabs


In [8]:
# Diviser les données en ensembles d'entraînement et de test
X_text_train, X_text_test, y_train, y_test = model_selection.train_test_split(movies_df['text'], movies_df['label'], test_size=0.2)
#vectorizer = CountVectorizer(ngram_range=(1,2))   #Mieux  en gardant les stopwords bizzare |bigram marche VRAIMENT bien, trigram impossible a faire pas assez de memoire :c ) 
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_text_train = vectorizer.fit_transform(X_text_train)
X_text_test = vectorizer.transform(X_text_test)

scaler = MinMaxScaler()
# Fit on training set only.
scaler.fit(X_text_train.toarray())

# Apply transform to both the training set and the test set.
X_text_train = scaler.transform(X_text_train.toarray())
X_text_test = scaler.transform(X_text_test.toarray())

pca = PCA(.99)

X_train_pca = pca.fit_transform(X_text_train)
X_test_pca = pca.transform(X_text_test)


In [10]:
model = clf = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000)
# Entraîner le modèle sur les données d'entraînement
model.fit(X_train_pca, y_train)

LogisticRegression(max_iter=1000)

In [11]:
fichier = open("./eval/testSentiment.txt", "r", encoding="utf-8")
contenu = fichier.read()
fichier.close()
eval_lines = contenu.split("\n")

In [13]:
X_eval = vectorizer.transform(eval_lines)

In [15]:
X_eval = scaler.transform(X_eval.toarray())

MemoryError: Unable to allocate 85.0 GiB for an array with shape (25001, 456364) and data type float64

In [16]:
X_eval_pca = pca.transform(X_eval)

In [ ]:
model.predict()

In [9]:
logistic_regression(X_text_train, X_text_test, y_train, y_test)

Accuracy :	0.8775
F1 score :	0.8836104513064132
AUC :		0.9428951864134258
